## dmuk-machine-learning

### BETTER Hackathon - Machine learning

Land cover changes and inter-annual vegetation performance analysis using ML algorithms

In [4]:
import os
import sys
sys.path.append(os.getcwd())
from dmuk_machine_learning_helpers import *
import geopandas as gpd
import getpass
from shapely.wkt import loads
from shapely.geometry import box
import cioppy

%load_ext autoreload
%autoreload 2

AttributeError: type object 'pandas._libs.tslibs.conversion._TSObject' has no attribute '__reduce_cython__'

### Authentication

In [2]:
if not 'username' in locals():
    username = input('What is your Terradue username?')
    %store username

KeyboardInterrupt: 

Set the API key associated to the Terradue username. 

It is available at this link: https://www.terradue.com/portal/settings/apikey (it requires a login with the Terradue username)

In [5]:
if not 'api_key' in locals():
    api_key = getpass.getpass('And your Terradue API key:')
    %store api_key
   

### Read the areas of interest

In [6]:
aoi_1 = gpd.read_file('data/BP_ex2_ROI_1.shp').iloc[0].geometry

aoi_1.wkt

'POLYGON ((9.858522328182319 56.38243630232368, 10.62722496475241 56.37473069755983, 10.61503107984585 56.08602974177386, 9.85208637694123 56.0936522354852, 9.858522328182319 56.38243630232368))'

In [40]:
aoi_2 = gpd.read_file('data/BP_ex2_ROI_2.shp').iloc[0].geometry

aoi_2.wkt

'POLYGON ((9.756741212054932 56.83325052692845, 10.43942854005751 56.8272456293202, 10.42951597141196 56.56666568192547, 9.75152837238649 56.57261168534836, 9.756741212054932 56.83325052692845))'

### Sentinel-2 Analysis Ready Data discovery

In [7]:
end_point = 'https://catalog.terradue.com/ard-s2-boa-reflectances/description'

Search parameters

In [96]:
search_params = dict()

search_params['start'] = '2019-04-01T23:59:59'
search_params['stop'] = '2019-04-30T23:59:59'
search_params['cat'] = '{!dataitem}'
search_params['geom'] = aoi_1.wkt

In [97]:
search_params

{'start': '2019-04-01T23:59:59',
 'stop': '2019-04-30T23:59:59',
 'cat': '{!dataitem}',
 'geom': 'POLYGON ((9.858522328182319 56.38243630232368, 10.62722496475241 56.37473069755983, 10.61503107984585 56.08602974177386, 9.85208637694123 56.0936522354852, 9.858522328182319 56.38243630232368))'}

In [99]:
ciop = cioppy.Cioppy()

search = gpd.GeoDataFrame(ciop.search(end_point=end_point,
                                      params=search_params,
                                      output_fields='self,enclosure,productType,cc,identifier,wkt,startdate,enddate,platform', 
                                      model='EOP', 
                                     creds=':'.join([username, api_key])))

search['geometry'] = search['wkt'].apply(loads)
search = search.drop(columns=['wkt'])
search

KeyError: 'wkt'

In [82]:
search.sample(5)

,self,enclosure,productType,cc,identifier,startdate,enddate,platform,wkt
1,https://catalog.terradue.com/ard-s2-boa-reflec...,https://store.terradue.com/ard-s2-boa-reflecta...,S2MSI2A-ARD,0.120914,BCBE514A81FA4F9A60DC078E3BFF5592F5BD3C59,2019-06-29T10:30:31.0240000Z,2019-06-29T10:30:31.0240000Z,S2A,"POLYGON((8.99967214264045 55.8448305261906,10...."
3,https://catalog.terradue.com/ard-s2-boa-reflec...,https://store.terradue.com/ard-s2-boa-reflecta...,S2MSI2A-ARD,60.24901,FC034541298F6638BA36C9FDD9BD1CCF960968D7,2019-06-25T10:50:31.0240000Z,2019-06-25T10:50:31.0240000Z,S2A,"POLYGON((8.99967214264045 55.8448305261906,10...."
12,https://catalog.terradue.com/ard-s2-boa-reflec...,https://store.terradue.com/ard-s2-boa-reflecta...,S2MSI2A-ARD,34.504602,27F4C7863EC2E7993875B05AFCD19E47BC30E93F,2019-06-10T10:50:39.0240000Z,2019-06-10T10:50:39.0240000Z,S2B,"POLYGON((8.99967214264045 55.8448305261906,10...."
2,https://catalog.terradue.com/ard-s2-boa-reflec...,https://store.terradue.com/ard-s2-boa-reflecta...,S2MSI2A-ARD,15.658536,0A9D31C75D3E0DAD6415264AC27DCA87E47328EB,2019-06-27T10:40:29.0240000Z,2019-06-27T10:40:29.0240000Z,S2B,"POLYGON((8.99967214264045 55.8448305261906,10...."
8,https://catalog.terradue.com/ard-s2-boa-reflec...,https://store.terradue.com/ard-s2-boa-reflecta...,S2MSI2A-ARD,48.973578,54AF66A034FDBD5661E8FAC9C34CDA905A631ACE,2019-06-17T10:40:29.0250000Z,2019-06-17T10:40:29.0250000Z,S2B,"POLYGON((8.99967214264045 55.8448305261906,10...."


In [94]:
product = search[search.startdate.str.contains('2019-06-10')].iloc[0]

In [95]:
product

self           https://catalog.terradue.com/ard-s2-boa-reflec...
enclosure      https://store.terradue.com/ard-s2-boa-reflecta...
productType                                          S2MSI2A-ARD
cc                                                     34.504602
identifier              27F4C7863EC2E7993875B05AFCD19E47BC30E93F
startdate                           2019-06-10T10:50:39.0240000Z
enddate                             2019-06-10T10:50:39.0240000Z
platform                                                     S2B
wkt            POLYGON((8.99967214264045 55.8448305261906,10....
Name: 12, dtype: object

In [46]:
src_ds = gdal.Open(get_vsi_url(product.enclosure, 
                               username,
                               api_key))

for band in range(src_ds.RasterCount):
    band += 1
    
    srcband = src_ds.GetRasterBand(band)
    print ('band index {} - {}'.format(band, srcband.GetDescription()))

band index 1 - B01
band index 2 - B02
band index 3 - B03
band index 4 - B04
band index 5 - B05
band index 6 - B06
band index 7 - B07
band index 8 - B08
band index 9 - B8A
band index 10 - B09
band index 11 - B11
band index 12 - B12
band index 13 - SCL
band index 14 - MSK_CLDPRB
band index 15 - MSK_SNWPRB


In [50]:
local_path = vsi_download(product.enclosure,
                          aoi_1,
                          username, 
                          api_key, 
                          bands = [2,3,4])

In [51]:
local_path

['data/subset_b2_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif',
 'data/subset_b3_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif',
 'data/subset_b4_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif']

In [8]:
'''
Merge bands and crop ROI.
'''

import subprocess
from osgeo import gdal

# Images to merge (each band)
input_blue_band = "data/subset_b2_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif"
input_blue_band
input_green_band = "data/subset_b3_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif"
input_red_band = "data/subset_b4_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif"
input_nir_band = "data/subset_b8_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif"

# Path to output merged image (with filename and .tif extrension).
output_image = "data/subset_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516_merge.tif"

# If you don't have the osgeo gdal_merge then you should have the gdal_merge command line tool
gm = '.conda/envs/env_dmuk_machine_learning_I/bin/gdal_merge.py'

merge_command = "python3.6 '{}' -o '{}' '{}' '{}' '{}' '{}'".format(gm,output_image, input_blue_band, input_green_band, input_red_band, input_nir_band)
merge_command
# This will now merge all the above bands and save the raster into ouput_image file
qsef = subprocess.Popen(merge_command,shell=True)
qsef.wait()
out, errs = qsef.communicate()

In [39]:
'''
Merge bands and crop ROI.
'''

import subprocess
from osgeo import gdal

# Images to merge (each band)
input_blue_band = "data/subset_b2_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif"
input_green_band = "data/subset_b3_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif"
input_red_band = "data/subset_b4_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif"
input_nir_band = "data/subset_b8_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif"

# Path to output merged image (with filename and .tif extrension).
output_image = "data/subset_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516_merge.tif"

# If you don't have the osgeo gdal_merge then you should have the gdal_merge command line tool
gm = '.conda/envs/env_dmuk_machine_learning_I/bin/gdal_merge.py'
python = '.conda/envs/env_dmuk_machine_learning_I/bin/python3'

#output = subprocess.check_output(["python3",gm,"-o",output_image,input_blue_band, input_green_band, input_red_band, input_nir_band],universal_newlines=True)
#output
command = [python,gm,"-o",output_image,input_blue_band, input_green_band, input_red_band, input_nir_band]
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

if result.returncode == 0:
    result.stdout
else:
    if result.stderr:
        print(result.stderr)

In [26]:
import subprocess,sys
from osgeo import gdal

# Path to input image for cropping (output of merge).
input_image = "subset_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516_merge.tif"

# Path to output cropped image (with filename and .tif extrension).
output_image = "subset_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516_merge_crop.tif"


# Path to crop mask (shapefile)
input_shpfile = 'data/BP_ex2_ROI_1.shp'

# Now use the gdal warp to crop the image using the defined shapefile.
output = subprocess.check_output(["gdalwarp","-cutline",input_shpfile,"-crop_to_cutline","-dstalpha",input_image,output_image],universal_newlines=True)

'Creating output file that is 4748P x 3215L.\nProcessing input file data/subset_b2_S2A_MSIL2A_20190629T103031_N0212_R108_T32VNH_20190629T130516.tif.\n0...10...20...30...40...50...60...70...80...90...100 - done.\n'

In [109]:
os.listdir()

['environment.yml',
 '.ipynb_checkpoints',
 '.conda',
 'dmuk_machine_learning_helpers.py',
 '__pycache__',
 'data',
 'environment_BETTER_AI.yaml',
 'README.md',
 'pom.xml',
 'scripts',
 'experiment.ipynb',
 'filename']

In [12]:
os.getcwd()

'/workspace'